## RSA - The Project

Name:  George Sorial

<hr />

# Table of Contents

### 1. Introduction  (5 points)

### 2. RSA Code Package    (10 points)
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.1 Basic tool set
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.2 First tool set
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; 2.3 Second tool set

### 3. RSA More Code  (10 points)

###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  Encode
###### &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;  Decode

### 4. Show a demo of encoding and decoding a message that highlights how your code works and the steps involved.   (20points)

### 5. Project Narrative AND results of exchanging codes with others.  (20 points)

### 6. Testing and comparing FME to mod.       (10 points)

### Points for 7 - 8 only available if the student is exchanging messages by the Friday (midnight) before the project is due.

### 7. Breaking codes without a private key    (10 points)

### 8. Advanced options (15 points) Needed to get 100%.
















### 1. Introduction to your RSA Package and Project.  Give an overview of your project and what you have learned. 

This is a collection of functions that allow the user to encode and decode messages using the RSA cryptosystem. By providing two prime numbers and a message, this package will provide a public key, private key, and encoded message. The user can also then decode messages they receive using the provided private key. 

I go into more detail on the purpose of each function below, but as an overview: the basic tool set includes functions to first convert a message into a list of ascii integers, as well as a function to do the reverse. There is also a function to convert an integer to its binary expansion, however, I opted to use a different strategy. The first tool set includes a function for Fast Modular Exponentiation to calculate the modulus of a number raised to a very high exponent. We will review why this is much more efficient than using the built in exponential operator later on. The other function in the first tool set is the Euclidean Algorithm. This calculates the greatest common divisor which is then used in the Find Public Keyh E function to find an appropriate public key. The final piece of this puzzle is the function Find Private Key D. This goes through the Extended Euclidean Algorithm to find the modular inverse used to decrypt messages. Finally with these functions in place, the user can use Decode and Encode messages.

Working through this project, I was able to apply the various algorithms and ideas from Discrete Structures into a functioning program. In terms of coding, I learned that syntax is everything. At one point, I kept trying to code a loop using C++ syntax. I found a quick refresher on python online and was back to it in no time.

### 2.1
#### Basic tool set

These are functions that you'll need to pre-process the messages before the messages are encoded and decoded by the RSA algorithm. That is the reason we will be defining them first.



In [1]:
def Convert_Text(_string):
    """
    Define this function such that it takes in a simple 
    string such as "hello" and outputs the corresponding
    standard list of integers (ascii) for each letter in the word hello.
    For example:
    _string = hello
    integer_list = [104, 101, 108, 108, 111]
    """
    integer_list = []
    for let in _string:
        integer_list.append(ord(let))
    return integer_list

In [2]:
def Convert_Num(_list):
    """
    Do the opposite of what you did in the Convert_Text
    function defined above.
    
    Define this function such that it takes in a list of integers
    and outputs the corresponding string (ascii).
    
    For example:
    _list = [104, 101, 108, 108, 111]
    _string = hello
    """
    _string = ''
    for i in _list:
        _string += chr(i)
    return _string

In [3]:
def Convert_Binary_String(_int):
    """
    Here, you need to define a function that converts an integer to
    a string of its binary expansion.
    
    For example:
    _int = 345
    bits = 101011001
    """
    bits=""
    remaining=_int
    if remaining==0:
        binary=[0]
    while (remaining>0):
        bits=str(remaining%2)+bits
        remaining=remaining//2
    return bits

Now that you're done with the basic toolset we'll move on to the first tool set which is actually involved in the RSA system.

### 2.2 
#### First tool set.



In [33]:
def FME(b, n, m):
    """
    1. Using the fast modular exponentiation algorithm,
    the below function should return b**n mod m.
    As described on page 254. (however, you may input the exponent and 
    then convert it to a string - the book verison imports the binary expansion)  
    2. You should use the function defined above
    Convert_Binary_String()
    3. Use this string to test which components are used in the calculation.
    4. Yes, there are many other ways to do this. 

***UPDATE*** You may use the function you developed in your problem set - but be sure it is your own work, commented, etc.. and change inputs as needed.
    """  
    result=1 # Starting at 1. This is a running product corresponding to the bits that equal 1. We ignore the bits that = 0
    square=b # Starting this variable at b, and we'll square it each iteration.
    while n>0: # This loop will divide n by 2 each iteration, so it will stop at 0 or less
        k=n%2  # This finds the binary digit of the least significant digit
        if k==1: # if the bit is one, then we want to include this in the result
            result=(result*square)%m # here, we multiply the current result by the current square and take the mod of that result. 
        n//=2 #Divide n by 2 each time to get the quotient rounded down.
        square=(square*square)%m   #update the current square value
       
    return result

In [34]:
def Euclidean_Alg(a, b):
    """
    1. Calculate the Greatest Common Divisor of a and b.
    
    2. Unless b==0, the result will have the same sign as b (so that when
    b is divided by it, the result comes out positive).
    The function must return a single integer 'x' which is
    the gcd of a and b.
    """
    a=abs(a) #This ensures we don't have to worry about negative values.
    b=abs(b)
    if b>a:  #Here we ensure that a will be the greater value. If it is not, we swap the values.
        temp=a
        a=b
        b=temp
    while(b>0): #This loop will run until n is less than or equal to zero. The loop will find the modulus k of a and b, and update the values per the algorithm discovered by Euclid.
        k=a%b
        a=b
        b=k
        
    return a

### 2.3
#### Second tool set

Here we will implement the meat of the RSA cryptosystem. The functions below will generate the public and private key pairs which will then be used to create a ciphertext using the public key and then decode the same using the pirvate key.



In [2]:
def Find_Public_Key_e(p, q):
    """
    Implement this function such that
    it takes 2 primes p and q.
    
    Use the gcd function that you have 
    defined before.
    
    The function should return 2 elements as follows:
    public key: n
    public key: e 
    
    
    HINT: this function will run a loop to find e such 
    that e is relatively prime to (p - 1) (q - 1) 
    and not equal to p or q.
    """
    n=p*q
    
    for i in range(2,((p-1)*(q-1)),1):
        if Euclidean_Alg(i,((p-1)*(q-1)))==1 and i!=p and i!=q:
            e=i
            return n,e 

In [36]:
def Find_Private_Key_d(e, p, q):
    """
    Implement this method
    to find the decryption exponent d such that
    d is the modular inverse of e. 
    
    This will use the Extended Euclidean Algorithm
    
    This function should return the following:
    d: the decryption component.
    """
    s1=1    #Here I initialize the variables s1,t1 and s2,t2 to the starting coefficients
    t1=0
    s2=0
    t2=1 
    n=(p-1)*(q-1)
    while(n>0): #This loop will run until n is less than or equal to zero
        k=e%n   #k is the remainder
        g=e//n  #g is the quotient
        e=n     #m is then updated to n
        n=k     #n is updated to the quotient
        s1temp=s2   #to update the coefficient variables, we need temporary variables
        t1temp=t2
        s2temp=s1-g*s2
        t2temp=t1-g*t2
        s1=s1temp   #here we update coefficient variables using the temporary variables
        t1=t1temp
        s2=s2temp
        t2=t2temp
    while (s1<0):           # this loop takes advantage of the fact that there is not one single unique inverse of a mod.
        s1+=((p-1)*(q-1))
    return s1

### 3.
#### Putting things all together.

1. In this part, you will define two functions `Encode` and `Decode` which will use the public and private keys that you calculated using the above 2 functions in the second toolset.
2. Using the public key, the `Encode` function will encode a message and generate the corresponding cipher_text.
3. Using the private key, the `Decode` function will decode a ciper_text and recover the original message.



In [37]:
def Encode(n, e, message):
    """
    Here, the message will be a string of characters.
    Use the function Convert_Text from 
    the basic tool set and get a list of numbers.
    
    Encode each of those numbers using n and e and
    return the encoded cipher_text.
    """
    cipher_text = []
    for let in Convert_Text(message): #here we will go through each letter of the text. We will run FME using the letter and the provided e and n. The result is added to a list.
        cipher_text.append(FME(let, e, n))
        
    return cipher_text

[8397, 8244, 4303, 4303, 4399, 8202, 8397, 4399, 10916, 3120, 4838]


In [38]:
def Decode(n, d, cipher_text):
    """
    Here, the cipher_text will be a list of integers.
    First, you will decrypt each of those integers using 
    n and d.
    Later, you will need to use the function Convert_Num, that converts the integers to a string, 
    from the basic toolset to recover the original message as a string. 
    
    """
    integers = []
    message = ''
    
    for i in cipher_text: #We will run FME using each integer in the list and the provided d and n to decode the message. This gives us the ascii values which are appended to a list.
        integers.append(FME(i,d,n))
    message=Convert_Num(integers) #finally, the ascii values are converted to text
    
    return message

PIZZA PARTY


### 4.  Demonstrate how your RSA works below using a mix of text and code blocks that make it clear to the reader how this works and why you chose to put it together the way you did:

* This is a step by step guide to using your code with a specific example that I can follow.

* Encode a message (including generating keys).

* Publish your public key and message to Piazza.

* Decode messages from Piazza.

* Test!





The goal is to calculate n, d, and e. With these three values we can encode and decode messages.

**Step 1**: Chose two prime numbers, p and q. For this example, we will use 67 and 43 though in practice, prime numbers with hundreds of digits are used. 

**Step 2**: Use Find_Public_Key_e to calculate n and e.  n is simply the product of the two prime numbers 

<code> n=p*q </code>

To find e, we use the euclidean algorithm to find a divisor of 67 and 43 that is also relatively prime to (p-1)x(q-1) and also not equal to p or q. I set up the loop to start at 2 and increase by 1 up to (p-1)x(q-1). For each integer, the loop checks if the GCD is one and not equal to p or q. After coding it this way, I realized there are a couple of other options for this loop. It can go in the reverse direction starting from (p-1)x(q-1), and it can also check random intgers between this range.

In [3]:
print (Find_Public_Key_e(67, 43))

NameError: name 'Euclidean_Alg' is not defined

**Step 3**: After finding n and e, the final piece we need is 'd'. We can plug in e, p, q into the function Find_Private_Key_d. This will go through the Extended Euclidean Algorithm to find Bezout's coefficients. Like the Euclidean algorithm, this function finds the GCD, but it also provides s1 which happens to be the modular inverse of e mod (p-1)x(q-1). This modular inverse is 'd' To prevent a negative result for 'd', there is a loop that will add the modulus to 'd' until the result is positive; this only works because modular inverses are not unique.

In [40]:
print(Find_Private_Key_d(5, 67, 43))

1109


**Step 5**: Now with n, e, and d figured out we will start to encode and decode messages. The encode function will take n, e, and a message then use Convert_Text to convert each letter into ascii. For each ascii integer, we will use Fast Modular Exponentiation to calculate $ascii^{e}mod n$. We now have a message comprised of encoded ascii integers!

In [41]:
print (Encode(2881, 5, "Watch any good shows lately?"))

[1420, 1177, 1775, 1837, 2642, 2306, 1177, 1733, 557, 2306, 83, 1486, 1486, 23, 2306, 20, 2642, 1486, 405, 20, 2306, 2146, 1177, 1775, 2188, 2146, 557, 1187]


**Step 6**: Finally, we can decode a response to the question above using the function Decode. Decode needs n, d, and the encoded message. For each ascii integer in the encoded message we will again use the FME function, but this time the eponent value is 'd': $ascii^{d}mod n$ We now have the decoded ascii values and just need to convert from ascii to text using the Convert_Num function. Let's decode the response below:

In [42]:
print(Decode(2881,1109,[1420, 2188, 2306, 2133, 1213, 20, 1775, 2306, 709, 30, 1733, 30, 20, 2642, 2188, 23, 2306, 405, 1177, 1775, 1837, 2642, 30, 1733, 83, 2306, 2054, 2293, 1213, 2188, 2188, 1733, 2803, 20, 2306, 220, 1177, 2640, 1324, 30, 1775, 2054, 2306, 1486, 1733, 2306, 385, 2188, 1775, 709, 2146, 30, 119, 286, 2306, 868, 2188, 1177, 2146, 2146, 557, 2306, 2146, 1486, 242, 2188, 23, 2306, 30, 1775, 286, 2306, 1947, 2306, 1775, 2642, 30, 1733, 2627, 2306, 1775, 2642, 30, 20, 2306, 1837, 2146, 1177, 20, 20, 2306, 2755, 2198, 1486, 1324, 1177, 1324, 2146, 557, 2306, 2642, 2188, 2146, 2755, 2188, 23, 2306, 2640, 2188, 2306, 1177, 2755, 2755, 2198, 2188, 1837, 30, 1177, 1775, 2188, 2306, 30, 1775, 2306, 2188, 242, 2188, 1733, 2306, 2640, 1486, 2198, 2188, 2306, 1775, 2642, 1177, 1733, 2306, 1947, 2306, 405, 1486, 1213, 2146, 23, 2306, 2642, 1177, 242, 2188, 2022, 2306, 1775, 1486, 1486, 286]))

We just finished watching "Queen's Gambit" on Netflix. Really loved it. I think this class probably helped me appreciate it even more than I would have, too.


If you create a custom main function or a script to read codes as your CUSTOM FEATURE, you may include it here with a large heading CUSTOM FEATURE:

### 5. **Insert Narrative (ONE OR TWO pages of meaningful narrative about your process - not including the examples) here in this block. **

* Describe the results of SPECIFIC exchanging keys and codes with classmates. (at least 3 complete examples).

* What was challenging? 

* Who helped you with your project? 

* Which resources were most helpful?

* What was your Best Mistake, (funniest? most frustrating? the one you learned the most from?)

* Again, be sure to include results and a discussion of specific exchanges with others.


I feel like every aspect of this project was challenging albeit rewarding for me. Everything from Fast Modular Exponentiation to the Euclidean Algorithm to the Extended Euclidean Algorithm was unlike anything I have ever studied before. Each step of each idea truly required me to thoroughly understand every concept beforehand. For example, to complete the Fast Modular Exponentiation function, I had to first grasp the concept of binary expansion and incorporate that into the algorithm. Working through the Extended Euclidean Algorithm was the most difficult task. Initially, it was difficult to keep track of the numerous variables and equations needed each step of the way. Fortunately, the mastery workbook along with Sriram's video provided some framwork to help keep everything straight.

Of course, each piece of this project was challenging to grasp initially, but what really tied everything together for me was the RSA infographic that Professor Stade provided. There was a lot of great discussion going on in Piazza as well. I admit I had not used Python for a while and needed to turn to Google for some reminders. The 'ord' function was one I had not used before. I also embarassingly kept trying to set up a loop using C++ syntax (i=0...) The best mistake I made was misreading the statement: "Use GCD to find *e* which should be relatively prime to (p-1)x(q-1)" I misread that final part of the sentence as "relatively prime to (p-1) and (q-1)" which is a very different idea. Exchanging keys and codes with classmates was an excellent way to finish the project and tie everything together. 

I translated professor Stade's question 
Public key (n,e) = (5251,3)
Private key (n,d) = (5251, 3403)
Encoded Message = [2128, 1150, 4250, 1349, 1262, 3336, 2371, 2497, 519, 1262, 1263, 1105, 3336, 1349, 1262, 2310, 1105, 3336, 4115, 762, 2405, 1263, 1105, 3336, 1262, 1349, 1150, 1105, 1262, 506, 1105, 1105, 4723, 2405, 2497, 519, 1262, 1974, 2371, 58, 1262, 519, 1105, 1349, 1262, 4839, 1150, 1105, 2497, 1262, 1974, 2371, 58, 762, 1262, 13, 4679, 1573, 1262, 4115, 2371, 2310, 1105, 1262, 2405, 3336, 1262, 4839, 2371, 762, 1560, 2405, 2497, 519, 3250]

"What song best describes the feeling you get when your RSA code is working?

And responded with 
[443, 2497, 58, 2405, 1349, 1262, 1263, 1974, 1262, 1685, 2371, 421, 2405, 2497, 519]

Inuit by Foxing

Michael Condon asked:
Public Key (n, e) = (3233,17)
Private Key (n, d) = (3233, 2753)
Encoded Message: 
[1486, 1369, 1992, 487, 2185, 2160, 1992, 281, 2185, 2160, 745, 1773, 1992, 612, 3179, 281, 690, 1992, 2185, 2235, 745, 487, 1992, 2185, 2235, 1313, 1992, 1230, 884, 2185, 281, 690, 1992, 884, 2185, 1992, 2570, 2160, 487, 1992, 1107, 2170, 3179, 281, 2170, 1992, 281, 2185, 2271, 612, 1632, 2235, 487, 1992, 1107, 2185, 2160, 745, 1773, 1992, 487, 2185, 2160, 1992, 2570, 2160, 487, 3216]

"If you could pick only one stock to buy which company would you buy?"

and I responded with  [1486, 1956, 2578, 1313, 1992, 1632, 745, 1107, 1632, 487, 1230, 1992, 2570, 1313, 1313, 2235, 1992, 1632, 1992, 1369, 1632, 2235, 1992, 2185, 1369, 1992, 1230, 884, 2185, 281, 690, 1230, 1992, 884, 2170, 1632, 884, 1992, 1313, 1542, 2170, 3179, 2570, 3179, 884, 1992, 3179, 1773, 3179, 2185, 1230, 487, 2235, 281, 2412, 1632, 884, 3179, 281, 1992, 2412, 3179, 1230, 690, 1992, 436, 1759]

"I've always been a fan of stocks that exhibit idiosyncratic risk :D" which is a bit of a code in itself. We'll see if he picks up on what I mean :D

I asked Public Key (n,e) = (2881, 5)
Private Key (n,d) = (2881, 1109)
[1420, 1177, 1775, 1837, 2642, 2306, 1177, 1733, 557, 2306, 83, 1486, 1486, 23, 2306, 20, 2642, 1486, 405, 20, 2306, 2146, 1177, 1775, 2188, 2146, 557, 1187]

"Watch any good shows lately?"

and Tyuler Kinkade responded with [1420, 2188, 2306, 2133, 1213, 20, 1775, 2306, 709, 30, 1733, 30, 20, 2642, 2188, 23, 2306, 405, 1177, 1775, 1837, 2642, 30, 1733, 83, 2306, 2054, 2293, 1213, 2188, 2188, 1733, 2803, 20, 2306, 220, 1177, 2640, 1324, 30, 1775, 2054, 2306, 1486, 1733, 2306, 385, 2188, 1775, 709, 2146, 30, 119, 286, 2306, 868, 2188, 1177, 2146, 2146, 557, 2306, 2146, 1486, 242, 2188, 23, 2306, 30, 1775, 286, 2306, 1947, 2306, 1775, 2642, 30, 1733, 2627, 2306, 1775, 2642, 30, 20, 2306, 1837, 2146, 1177, 20, 20, 2306, 2755, 2198, 1486, 1324, 1177, 1324, 2146, 557, 2306, 2642, 2188, 2146, 2755, 2188, 23, 2306, 2640, 2188, 2306, 1177, 2755, 2755, 2198, 2188, 1837, 30, 1177, 1775, 2188, 2306, 30, 1775, 2306, 2188, 242, 2188, 1733, 2306, 2640, 1486, 2198, 2188, 2306, 1775, 2642, 1177, 1733, 2306, 1947, 2306, 405, 1486, 1213, 2146, 23, 2306, 2642, 1177, 242, 2188, 2022, 2306, 1775, 1486, 1486, 286]

"We just finished watching "Queen's Gambit" on Netflix. Really loved it. I think this class probably helped me appreciate it even more than I would have, too."

### 6. FME Exploration - (CODE INTERVIEW type question)



* Explain in detail (give an example) why RSA needs to use an FME function and cannot just simply use the Python Mod function % .

* You may use your arguments from the Number theory Mastery Workbook.

* You may wish to use an example from your code breaking experience.


As I mentioned above, RSA uses very high exponent values. The runtime using python's built in mathematical operators is prohibitively long as the number of digits in the exponent increases. Below you can see the brute force fakeFME function has a total runtime that is several orders of magnitude longer than the FME function below it. As the value of the exponent increases, the difference in runtime becomes more drastic.

In [43]:
%%time 
#I learned about this timer from a student on piazza, but I cannot find the post to properly cr
def fakeFME(b, n, m):
    return (b**n)%m
print(fakeFME(15,12345678,222))


147
CPU times: user 20.4 s, sys: 108 ms, total: 20.5 s
Wall time: 21.3 s


In [44]:
%%time
print (FME(15,12345678,222))

147
CPU times: user 862 µs, sys: 0 ns, total: 862 µs
Wall time: 398 µs


## CODE BREAKING ##

This section of the project is only available to students exchanging codes by the Friday (midnight) before the project is due.

After you have tested your RSA package yourself, and tested it with classmates by publishing both the private and public keys on Piazza, post 2 messages with just the public keys for people to break on the "Just Public Keys" thread - use both very small n ‘s (under 1000) for practice, and some with a challenge.


**Implementing a factoring algorithm:**

Begin by coding the basic brute force factorization algorithm given in pseudocode below.

Brute Force Factoring
<pre><code>def factorize(n):
    # n is a number, return the smallest factor of n
    for i from 2 to n-1:
        if i divides n:
            return i
        return FALSE
        </code></pre>

In [45]:
def factorize(n):
    for i in range(2,n-1): #this loop will return the first evenly divisble number between two and n-1
        if n%i==0: 
            return i, int(n/i)
    return False

### 7. Do the following for full credit in number 7
* To break others’ code you just need to factor n (public key). **Explain how and why code breaking works in a text block.**

* Demonstrate 1 example of breaking a code with a small n, and show the steps involved.

* Now attempt codes with larger n

* Describe your attempt to break codes with large n. At what values would you say n's get difficult? (too large to factor)

* Show COMPLETELY at least 3 codes you broke and/or codes of your that were broken.

The public key is *public* meaning that it's freely available for anyone to see. If 'n' is too small, then it takes less time for a program to factor it. Factoring 'e' gives you 'p' and 'q' which can lead you to 'd'. 'n' is typically 100s of digits long in practice, which is impossible to factorize in a reasonable amount of time. Below, factorizing an 18 digit 'n' takes approximately 25 seconds 

In [46]:
%%time
# in this example, we have a small value for n and e. n=703 and e=113
print(factorize(703)) #as you can see this gives us both p and q 

(19, 37)
CPU times: user 319 µs, sys: 0 ns, total: 319 µs
Wall time: 213 µs


In [47]:
#now let's find d by plugging in e, p, and q!
print(Find_Private_Key_d(113, 19, 37))

281


In [48]:
#Finally we'll decode the encoded message using n, d, and the cipher text
print(Decode(703,281,[544, 472, 233, 242, 250, 555, 591, 550, 376, 364, 210, 165, 526, 47, 555, 95, 476, 210, 242, 165, 95, 233, 526, 210, 376, 95, 233, 242, 364, 210, 242, 224, 376, 95, 364, 233, 47, 242, 210, 555, 376, 165, 472, 242, 555, 410, 242, 250, 95, 233, 233, 47, 145]))

The Conquistador's treasure is buried south of Creed.


The second code we'll break has a 7 digit 'n': 
Andrew Hogan
Public Key (n, e) -- 4182767, 1690319

Cipher: [2187474, 2164060, 1814081, 2164060, 1524456, 2517607, 1814081, 2517607, 979048, 3110313, 2517607, 518292, 4086418, 527853, 1814081, 4086418, 1541372, 2517607, 1814081, 1772795, 3110313, 778876, 1524456, 745612, 527853, 1541372, 1814081, 347101, 1524456, 847187, 164557, 745612, 527853, 745612, 4086418, 745612, 2164060, 1524456, 313892]

In [49]:
%%time
print(factorize(4182767)) # this doesn't take much longer to compute even though I'm using a brute force algorithm. At 7 digits, it still won't take the 'eoons' it can with higher digits values.

(1031, 4057)
CPU times: user 260 µs, sys: 0 ns, total: 260 µs
Wall time: 170 µs


In [50]:
print(Find_Private_Key_d(1690319,1031,4057)) #get d using e,p,and q. Then decode
print(Decode(4182767, 3567839, [2187474, 2164060, 1814081, 2164060, 1524456, 2517607, 1814081, 2517607, 979048, 3110313, 2517607, 518292, 4086418, 527853, 1814081, 4086418, 1541372, 2517607, 1814081, 1772795, 3110313, 778876, 1524456, 745612, 527853, 1541372, 1814081, 347101, 1524456, 847187, 164557, 745612, 527853, 745612, 4086418, 745612, 2164060, 1524456, 313892]))
print(Encode(4182767,1690319, "NII")) #I'll reply in the most annoying way possible :D


3567839
No one expects the Spanish Inquisition.
[2187474, 347101, 347101]


Here's one with an 18 digit 'n'
Bryan Bender 4 days ago
Public Key: [193604375590086907, 3]

Cipher: [274625, 1331000, 1771561, 1367631, 1331000, 1030301, 32768, 1685159, 1124864, 1367631, 32768, 912673, 1560896, 1560896, 1030301, 1295029, 1404928, 1560896, 1520875, 32768, 1560896, 1367631, 32768, 1092727, 1030301, 1331000, 1030301, 1481544, 912673, 1560896, 1030301, 32768, 1481544, 912673, 1331000, 1000000, 1367631, 1295029, 32768, 1331000, 1601613, 1295029, 941192, 1030301, 1481544, 1520875, 32768, 941192, 1771561, 32768, 1000000, 1030301, 1560896, 1030301, 1481544, 1295029, 1157625, 1331000, 1157625, 1520875, 1560896, 1157625, 970299, 32768, 1295029, 1030301, 912673, 1331000, 1520875, 32768, 1157625, 1520875, 85184, 32768, 1367631, 1061208, 32768, 970299, 1367631, 1601613, 1481544, 1520875, 1030301, 85184, 32768, 1259712, 1157625, 1643032, 1157625, 1331000, 1092727, 32768, 1157625, 1331000, 32768, 912673, 32768, 1520875, 1560896, 912673, 1560896, 1030301, 32768, 1367631, 1061208, 32768, 1520875, 1157625, 1331000, 97336]

In [51]:
%%time
print(factorize(193604375590086907)) #This takes considerably longer to computer, although the wait is worth it if we're trying to decipher a juicy message. 

(220004399, 880002293)
CPU times: user 23.7 s, sys: 0 ns, total: 23.7 s
Wall time: 25 s


In [52]:
print(Find_Private_Key_d(3,220004399,880002293)) # gives us d of 129069582993386811 which we plug into decode 
print(Decode(193604375590086907, 129069582993386811, [274625, 1331000, 1771561, 1367631, 1331000, 1030301, 32768, 1685159, 1124864, 1367631, 32768, 912673, 1560896, 1560896, 1030301, 1295029, 1404928, 1560896, 1520875, 32768, 1560896, 1367631, 32768, 1092727, 1030301, 1331000, 1030301, 1481544, 912673, 1560896, 1030301, 32768, 1481544, 912673, 1331000, 1000000, 1367631, 1295029, 32768, 1331000, 1601613, 1295029, 941192, 1030301, 1481544, 1520875, 32768, 941192, 1771561, 32768, 1000000, 1030301, 1560896, 1030301, 1481544, 1295029, 1157625, 1331000, 1157625, 1520875, 1560896, 1157625, 970299, 32768, 1295029, 1030301, 912673, 1331000, 1520875, 32768, 1157625, 1520875, 85184, 32768, 1367631, 1061208, 32768, 970299, 1367631, 1601613, 1481544, 1520875, 1030301, 85184, 32768, 1259712, 1157625, 1643032, 1157625, 1331000, 1092727, 32768, 1157625, 1331000, 32768, 912673, 32768, 1520875, 1560896, 912673, 1560896, 1030301, 32768, 1367631, 1061208, 32768, 1520875, 1157625, 1331000, 97336]))


129069582993386811
Anyone who attempts to generate random numbers by deterministic means is, of course, living in a state of sin.



## Custom Feature ##
The last 15 points of the require a custom feature in your project. This is an opportinity for you to explore an RSA or coding topic of interest to you. At this point it is not about points, but exploring advanced ideas relevant to you.  This is also a feature that should be relevant to your programming ability. If you are in 1300, creating a main type function may be enough. If you are an experienced programmer, find a topic to push yourself. 


Custom Features could one or more of the following:

1. A Python "main" function that allows a user to "Get keys/Encode/Decode/Break Codes" etc... 
2. An exploration of factoring improvements and analyze how effective they are. Must include some kind of timing analysis and multiple code breaking. A superficial treatment will not recieve full points. Must be throughly tested with excellent investigations to recieve full points.
3. A script to read codes from Piazza data. You can copy and paste codes from Piazza into a txt files and then have your script read it. This is a VERY helpful tool for doing the project as well. 
4. Exploration of advanced factoring alogrithms (see Moodle) Try Pollard's Rho <- super satisfying. (full points if well - impemented with thoughtful examples)
5. Advanced complexity analysis. 
6. Other options available, please ask. 

The Custom Feature is an important part of the project, and it should be a non-trivial response. 
Please help organize your Custom Feature by breaking it up into sections - ie. not one long block of text.
Mininimally executed, "dialling it in" type features will not recieve full points. 
See below for more detials on grading of this section.



 




In [3]:
https://home.cs.colorado.edu/~srirams/courses/csci2824-spr14/pollardsRho.html"""%%time  
def pollard(n):
    a = 2
    b = 3 #starts at 2 in example 
    p=1
    while (b!=a): #maybe while p==1?
        a = (a**2+2)%n
        b = ((b**2+2)%n)**2+2%n
        p = Euclidean_Alg( abs(a-b) , n)
        print(a,b,p)
    return p
print(pollard(4182767))
##ran out of time for this. in it's current state, it is way too slow"""

SyntaxError: invalid syntax (1094938676.py, line 1)

**Please use the grading guide below to self grade your Custom Feature.**

GRADING FOR CUSTOM FEATURE:

15 pts  Wow.  It is amazing.
        Shows initiative and originality. 
        You did something extra special and pushed yourself.
        You went beyond all expectations 
        You broke the rules in a creative way. 
        Your coding and commenting is exceptional.
        Excellent 

10 - 14 points - It is GOOD! 
        You were a “self starter”. 
         You did everything  requested. 
        All expectations met. 
       	You did a very good job. 
        Good use of commenting.
        Shows mastery of skills.

5 -10 points  - Okay.
        Minimum requirements.
        Commenting weak.

0 -5 points - It is not finished. 
          Does meet objectives. 
            Did not follow directions. 
            Choose not to do this part (which is a totally fine choice)


## Self grading of custom feature ##

Here in this block

Please rate what score out of 15 your custom feature merits given the criteria above:

* For example, if you are brand new to programming, using a main function could be an amazing feature - tell me how this helped you move to a new level.
* In what ways did YOU push yourself? Did you try something you've never done before?
* Alternatively, if you just did not have the time to do the custum feature, that is actually just fine and a reasonable choice for an adult. No judgment.

## Grading Notes ##

*The project will consist of a combination of text and code blocks.

*Grading - all code must be well commented. These should be easy to read. Here is a simple guide: https://realpython.com/python-comments-guide/ 

*See notes in Moodle about comments and this guide (You do need to have comments on ALL functions and key routines in your code.

You also need to use the variables given (unless you are restructuring a code copy for your custom feature) - so notice this guide says you can remove comments only in specific contexts.  

For our Project make sure you have adequate comments that show you understand the code you are using.)

*The narrative elements of the project are just as important as the coding.

*Do NOT copy code from the internet (or anywhere)

*Do not ask questions on stackoverflow or similar etc…

*You are expected to have read the honesty notes in the syllabus.

*Credit sources you used for additional understanding.

*Credit classmates that helped you.

